In [1]:
import sys
sys.path.append('/home/samer/projects/fuzzy_sql/src') #This will enable reading the modules
from pathlib import Path
import os
import json

from fuzzy_sql.rnd_query import RND_QUERY
from fuzzy_sql.fuzzy_sql import *

In [2]:
#set directories
root_dir=Path('/home/samer/projects/fuzzy_sql')
metadata_dir=os.path.join(root_dir,'data/sdgd/processed/metadata')
db_path=os.path.join(root_dir,'db/sdgd.db')

In [3]:
#define input tables and metadata

tbl_names_lst=["C1"]

metadata_lst=[]
for tbl_name in tbl_names_lst:
    with open(os.path.join(metadata_dir,tbl_name+'.json'),'r') as f:
        metadata_lst.append(json.load(f))

In [4]:
#connect to db
conn = sqlite3.connect(db_path)
self=RND_QUERY(conn, tbl_names_lst, metadata_lst)

Metadata for table C1 is valid.
Parameter input is valid.


# Generate Random  queries 

In [5]:
#Single Aggregate

rnd_query=self.make_single_agg_query()


n_rows=rnd_query['query_desc']['n_rows']
n_cols=rnd_query['query_desc']['n_cols']

print(f"no_of_rows={n_rows}\nno_of_cols={n_cols}\nproduct={n_rows*n_cols/1000}")
print("\n")
print(rnd_query['query_desc']['sql'])

no_of_rows=3339
no_of_cols=8
product=26.712


SELECT workclass, relationship, marital_status, education, income, race, COUNT(*), MAX(age)  FROM C1  GROUP BY workclass, relationship, marital_status, education, income, race


In [6]:
rnd_query['query_desc']

{'type': 'single_agg',
 'agg_fntn': ('MAX', 'age'),
 'grpby_vars': ['workclass',
  'relationship',
  'marital_status',
  'education',
  'income',
  'race'],
 'from_tbl_name': 'C1',
 'join_tbl_name_lst': [],
 'sql': 'SELECT workclass, relationship, marital_status, education, income, race, COUNT(*), MAX(age)  FROM C1  GROUP BY workclass, relationship, marital_status, education, income, race',
 'n_rows': 3339,
 'n_cols': 8}

In [7]:
# Twin Aggregate 

syn_tbl_name_lst=['C1_syn_default_1']
rnd_query=self.make_twin_agg_query(syn_tbl_name_lst)

n_rows_real=rnd_query['query_desc']['n_rows_real']
n_cols_real=rnd_query['query_desc']['n_cols_real']
n_rows_syn=rnd_query['query_desc']['n_rows_syn']
n_cols_syn=rnd_query['query_desc']['n_cols_syn']

print(rnd_query['query_desc']['sql_real'])
print(f"REAL:\nno_of_rows={n_rows_real}\nno_of_cols={n_cols_real}\nproduct={n_rows_real*n_cols_real/1000}\n\n")

print(rnd_query['query_desc']['sql_syn'])
print(f"SYN:\nno_of_rows={n_rows_syn}\nno_of_cols={n_cols_syn}\nproduct={n_rows_syn*n_cols_syn/1000}\n\n")



SELECT native_country, income, occupation, COUNT(*), SUM(age)  FROM C1  GROUP BY native_country, income, occupation
REAL:
no_of_rows=718
no_of_cols=5
product=3.59


SELECT native_country, income, occupation, COUNT(*), SUM(age)  FROM C1_syn_default_1  GROUP BY native_country, income, occupation
SYN:
no_of_rows=716
no_of_cols=5
product=3.58




In [8]:
rnd_query['query_desc']

{'type': 'twin_agg',
 'agg_fntn': ('SUM', 'age'),
 'grpby_vars': ['native_country', 'income', 'occupation'],
 'from_tbl_name_real': 'C1',
 'join_tbl_name_lst_real': [],
 'sql_real': 'SELECT native_country, income, occupation, COUNT(*), SUM(age)  FROM C1  GROUP BY native_country, income, occupation',
 'n_cols_real': 5,
 'n_rows_real': 718,
 'from_tbl_name_syn': 'C1_syn_default_1',
 'join_tbl_name_lst_syn': [],
 'sql_syn': 'SELECT native_country, income, occupation, COUNT(*), SUM(age)  FROM C1_syn_default_1  GROUP BY native_country, income, occupation',
 'n_cols_syn': 5,
 'n_rows_syn': 716}

In [9]:
#Single Filter

# self.no_groupby_vars=2
# self.no_where_vars=2
# self.max_in_terms=2

rnd_query=self.make_single_fltr_query()


n_rows=rnd_query['query_desc']['n_rows']
n_cols=rnd_query['query_desc']['n_cols']

print(f"no_of_rows={n_rows}\nno_of_cols={n_cols}\nproduct={n_rows*n_cols/1000}")
print("\n")
print(rnd_query['query_desc']['sql'])

no_of_rows=0
no_of_cols=13
product=0.0


SELECT *  FROM C1  WHERE   fnlwgt = 192203 AND  relationship <> 'Husband' AND NOT  hours_per_week > 8 AND  education IN ('10th', 'Bachelors', 'Some-college') AND  capital > 0 AND  occupation IN ('Craft-repair', 'Transport-moving') 


In [10]:
rnd_query['query_desc']

{'type': 'single_fltr',
 'from_tbl_name': 'C1',
 'join_tbl_name_lst': [],
 'sql': "SELECT *  FROM C1  WHERE   fnlwgt = 192203 AND  relationship <> 'Husband' AND NOT  hours_per_week > 8 AND  education IN ('10th', 'Bachelors', 'Some-college') AND  capital > 0 AND  occupation IN ('Craft-repair', 'Transport-moving') ",
 'n_rows': 0,
 'n_cols': 13}

In [11]:
# Twin Filter 


syn_tbl_name_lst=['C1_syn_default_1']
rnd_query=self.make_twin_fltr_query(syn_tbl_name_lst)

n_rows_real=rnd_query['query_desc']['n_rows_real']
n_cols_real=rnd_query['query_desc']['n_cols_real']
n_rows_syn=rnd_query['query_desc']['n_rows_syn']
n_cols_syn=rnd_query['query_desc']['n_cols_syn']

print(f"REAL:\nno_of_rows={n_rows_real}\nno_of_cols={n_cols_real}\nproduct={n_rows_real*n_cols_real/1000}\n\n")
print(f"SYN:\nno_of_rows={n_rows_syn}\nno_of_cols={n_cols_syn}\nproduct={n_rows_syn*n_cols_syn/1000}\n\n")
print(rnd_query['query_desc']['sql_real'])
print("\n")
print(rnd_query['query_desc']['sql_syn'])

REAL:
no_of_rows=3454
no_of_cols=13
product=44.902


SYN:
no_of_rows=3265
no_of_cols=13
product=42.445


SELECT *  FROM C1  WHERE   race <> 'White' AND  relationship IN ('Husband', 'Not-in-family') AND NOT  age = 48 AND  education <> 'Assoc-acdm' 


SELECT *  FROM C1_syn_default_1  WHERE   race <> 'White' AND  relationship IN ('Husband', 'Not-in-family') AND NOT  age = 48 AND  education <> 'Assoc-acdm' 


In [12]:
rnd_query['query_desc']

{'type': 'twin_fltr',
 'from_tbl_name_real': 'C1',
 'join_tbl_name_lst_real': [],
 'sql_real': "SELECT *  FROM C1  WHERE   race <> 'White' AND  relationship IN ('Husband', 'Not-in-family') AND NOT  age = 48 AND  education <> 'Assoc-acdm' ",
 'n_cols_real': 13,
 'n_rows_real': 3454,
 'from_tbl_name_syn': 'C1_syn_default_1',
 'join_tbl_name_lst_syn': [],
 'sql_syn': "SELECT *  FROM C1_syn_default_1  WHERE   race <> 'White' AND  relationship IN ('Husband', 'Not-in-family') AND NOT  age = 48 AND  education <> 'Assoc-acdm' ",
 'n_cols_syn': 13,
 'n_rows_syn': 3265}

In [13]:
#Single Aggregate-Filter

rnd_query=self.make_single_aggfltr_query()


n_rows=rnd_query['query_desc']['n_rows']
n_cols=rnd_query['query_desc']['n_cols']

print(f"no_of_rows={n_rows}\nno_of_cols={n_cols}\nproduct={n_rows*n_cols/1000}")
print("\n")
print(rnd_query['query_desc']['sql'])

no_of_rows=920
no_of_cols=11
product=10.12


SELECT race, education, relationship, workclass, income, occupation, sex, marital_status, native_country, COUNT(*), SUM(fnlwgt)  FROM C1  WHERE   marital_status IN ('Divorced', 'Never-married') AND  sex <> 'Male' AND  hours_per_week > 40  GROUP BY race, education, relationship, workclass, income, occupation, sex, marital_status, native_country


In [14]:
# Twin Aggregate-Filter 

syn_tbl_name_lst=['C1_syn_default_1']
rnd_query=self.make_twin_aggfltr_query(syn_tbl_name_lst)

n_rows_real=rnd_query['query_desc']['n_rows_real']
n_cols_real=rnd_query['query_desc']['n_cols_real']
n_rows_syn=rnd_query['query_desc']['n_rows_syn']
n_cols_syn=rnd_query['query_desc']['n_cols_syn']

print(rnd_query['query_desc']['sql_real'])
print(f"REAL:\nno_of_rows={n_rows_real}\nno_of_cols={n_cols_real}\nproduct={n_rows_real*n_cols_real/1000}\n\n")

print(rnd_query['query_desc']['sql_syn'])
print(f"SYN:\nno_of_rows={n_rows_syn}\nno_of_cols={n_cols_syn}\nproduct={n_rows_syn*n_cols_syn/1000}\n\n")

SELECT sex, marital_status, education, income, workclass, relationship, occupation, COUNT(*), AVG(age)  FROM C1  WHERE  NOT  capital > 0 AND  occupation LIKE 'Adm-clerical' AND  sex NOT LIKE 'Female' AND  race NOT LIKE 'White'  GROUP BY sex, marital_status, education, income, workclass, relationship, occupation
REAL:
no_of_rows=115
no_of_cols=9
product=1.035


SELECT sex, marital_status, education, income, workclass, relationship, occupation, COUNT(*), AVG(age)  FROM C1_syn_default_1  WHERE  NOT  capital > 0 AND  occupation LIKE 'Adm-clerical' AND  sex NOT LIKE 'Female' AND  race NOT LIKE 'White'  GROUP BY sex, marital_status, education, income, workclass, relationship, occupation
SYN:
no_of_rows=90
no_of_cols=9
product=0.81




In [15]:
matched_query=self._match_twin_query(rnd_query)
scored_query=self.calc_dist_scores(matched_query)

In [16]:
scored_query['query_desc']

{'type': 'twin_agg',
 'agg_fntn': ('AVG', 'age'),
 'grpby_vars': ['sex',
  'marital_status',
  'education',
  'income',
  'workclass',
  'relationship',
  'occupation'],
 'from_tbl_name_real': 'C1',
 'join_tbl_name_lst_real': [],
 'sql_real': "SELECT sex, marital_status, education, income, workclass, relationship, occupation, COUNT(*), AVG(age)  FROM C1  WHERE  NOT  capital > 0 AND  occupation LIKE 'Adm-clerical' AND  sex NOT LIKE 'Female' AND  race NOT LIKE 'White'  GROUP BY sex, marital_status, education, income, workclass, relationship, occupation",
 'n_cols_real': 9,
 'n_rows_real': 115,
 'from_tbl_name_syn': 'C1_syn_default_1',
 'join_tbl_name_lst_syn': [],
 'sql_syn': "SELECT sex, marital_status, education, income, workclass, relationship, occupation, COUNT(*), AVG(age)  FROM C1_syn_default_1  WHERE  NOT  capital > 0 AND  occupation LIKE 'Adm-clerical' AND  sex NOT LIKE 'Female' AND  race NOT LIKE 'White'  GROUP BY sex, marital_status, education, income, workclass, relationship, 

In [17]:
scored_query['query_hlngr_score']

0.5923274472978842

In [18]:
scored_query['query_ecldn_score']

0.0